We rarely get our data in just the form we want.  In this worksheet we will investigate some techniques for making relatively simple adjustments that help us get _from our input data into a data frame that is ready for us to use_.

We have some data in the file `iris.csv` which has been hand-transcribed from notebooks and contains a few errors.  To clean the data we will:
  * Load up the data
  * Understand the Data
  * Remove empty values
  * Fix format errors
  * Fix incorrect data
  * Remove duplicates

After this, we are ready to visualise the data (which we will learn more about next week).

In [1]:
import pandas as pd

# load data - notice that iris data has no index, so we will use a fresh one
iris = pd.read_csv("data/iris.csv")
iris


,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
147,6.7,3.0,5.2,2.3,Iris-virginica
148,6.3,2.5,5.0,1.9,Iris-virginica
149,6.5,3.0,5.2,2.0,Iris-virginica
150,6.2,3.4,5.4,2.3,Iris-virginica


Jupyter always gives us a preview of our data when we print it, but we can specifially ask for the `head` rows, the `tail` rows and `info` about any dataframe.  `info` in particular gives us very useful information about our data.  Pay close attention to the "non-null" count and the data type of each column.


In [2]:
import pandas as pd

# load data - notice that iris data has no index, so we will use a fresh one
iris = pd.read_csv("data/iris.csv")
iris


,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
147,6.7,3.0,5.2,2.3,Iris-virginica
148,6.3,2.5,5.0,1.9,Iris-virginica
149,6.5,3.0,5.2,2.0,Iris-virginica
150,6.2,3.4,5.4,2.3,Iris-virginica


In [3]:
print(iris.head(2))
print("")
print(iris.tail(2))
print("")
print(iris.info())

   sepal length  sepal width  petal length  petal width        class
0           5.1          3.5           1.4          0.2  Iris-setosa
1           4.9          3.0           1.4          0.2  Iris-setosa

     sepal length  sepal width  petal length  petal width           class
150           6.2          3.4           5.4          2.3  Iris-virginica
151           5.9          3.0           5.1          1.8  Iris-virginica

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal length  152 non-null    float64
 1   sepal width   151 non-null    float64
 2   petal length  152 non-null    float64
 3   petal width   152 non-null    float64
 4   class         152 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.1+ KB
None




The first issue that shows up is the missing value in "sepal width".  Every other column has 152 values, but this one has just 151.  Missing values in a table are called "null" values and they can mess with our analysis if we are not aware of them.  Sometimes we do want to leave them there, but often we want to exclude that data.

**YOU MUST NEVER MODIFY YOUR SOURCE DATA**

If there is "junk" in your input file, never make changes directly in the file, for the following reasons:
  * data is often audited and modification of official data could be an infringement
  * one person's junk is another person's treasure
  * pandas can adjust the data for you easily so you can have a "clean" version without adjusting the original

Lets start by finding that null value.  We know it is in the "sepal width" series


In [4]:
iris["sepal width"]

0      3.5
1      3.0
2      3.2
3      3.1
4      3.6
      ... 
147    3.0
148    2.5
149    3.0
150    3.4
151    3.0
Name: sepal width, Length: 152, dtype: float64

we can't see it in the preview, but pandas can filter a series to keep only the null values, but do that, we will need to understand indexing with masks.   

Firstly, we get the "sepal width" series from the data frame.  Then we call a function on that series which will convert all values to true or false.  If the original value was not null, we will get false, if it was null, we get true.  You will have to believe me, but there is a single "True" in there.

In [5]:
mask = iris["sepal width"].isnull()

print(mask.info())

mask

<class 'pandas.core.series.Series'>
RangeIndex: 152 entries, 0 to 151
Series name: sepal width
Non-Null Count  Dtype
--------------  -----
152 non-null    bool 
dtypes: bool(1)
memory usage: 280.0 bytes
None


0      False
1      False
2      False
3      False
4      False
       ...  
147    False
148    False
149    False
150    False
151    False
Name: sepal width, Length: 152, dtype: bool

Recall that we call a series that is all booleans a "mask". Only indexes with a "True" in the mask result in a row being included in the result.

In [6]:
iris[mask]

,sepal length,sepal width,petal length,petal width,class
128,6.2,NaN,4.8,1.8,Iris-virginica


and thus we can see our problem.  There is a number missing in row 128.  Check the original dataframe to see for yourself.  We can also use `loc` or `iloc` to see the rows in this vicinity now we have identified the problem

In [7]:
iris.loc[120:130,:]

,sepal length,sepal width,petal length,petal width,class
120,7.7,2.6,6.9,2.3,Iris-virginica
121,6.0,2.2,5.0,1.5,Iris-virginica
122,6.9,3.2,5.7,2.3,Iris-virginica
123,5.6,2.8,4.9,2.0,Iris-virginica
124,7.7,2.8,6.7,2.0,6.3
125,6.3,2.7,4.9,1.8,Iris-virginica
126,6.7,3.3,5.7,2.1,Iris-virginica
127,7.2,3.2,6.0,1.8,Iris-virginica
128,6.2,NaN,4.8,1.8,Iris-virginica
129,6.1,3.0,4.9,1.8,Iris-virginica


# Exercise - wide petals

Identify all rows where the "petal width" is greater than 5.  How many are there?

In [9]:
print("put your answer in this code block")
large_petals = iris['petal width'] > 5
display(iris[large_petals])

put your answer in this code block


,sepal length,sepal width,petal length,petal width,class
134,6.4,2.8,5.6,22.0,Iris-virginica


## Fixing the error

What do to about the error is up to you.  You should create a "clean" data frame, separate to the other one regardless of your decision.  Possible choices are:
  * remove that whole row
  * remove that whole column
  * choose a value for the missing entry

We will demonstrate each.


In [10]:
# remove that row

# we need the opposite mask. A trick to do this is to perform the "equals false" operation :)
mask2 = mask == False

# we can then use this as a mask to get only the rows we want.
clean_iris = iris[mask2]
clean_iris

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
147,6.7,3.0,5.2,2.3,Iris-virginica
148,6.3,2.5,5.0,1.9,Iris-virginica
149,6.5,3.0,5.2,2.0,Iris-virginica
150,6.2,3.4,5.4,2.3,Iris-virginica


In [25]:
# alternative!  Since we know exactly what row to drop, we can use the drop function

clean_iris = iris.drop(index=126)
clean_iris

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
147,6.7,3.0,5.2,2.3,Iris-virginica
148,6.3,2.5,5.0,1.9,Iris-virginica
149,6.5,3.0,5.2,2.0,Iris-virginica
150,6.2,3.4,5.4,2.3,Iris-virginica


In [26]:
# alternative - dropna does _all_ the hard work for us
clean_iris = iris.dropna()
clean_iris

,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
147,6.7,3.0,5.2,2.3,Iris-virginica
148,6.3,2.5,5.0,1.9,Iris-virginica
149,6.5,3.0,5.2,2.0,Iris-virginica
150,6.2,3.4,5.4,2.3,Iris-virginica


In [27]:
# remove whole column with drop

clean_iris = iris.drop(columns="sepal width")
clean_iris

,sepal length,petal length,petal width,class
0,5.1,1.4,0.2,Iris-setosa
1,4.9,1.4,0.2,Iris-setosa
2,4.7,1.3,0.2,Iris-setosa
3,4.6,1.5,0.2,Iris-setosa
4,5.0,1.4,0.2,Iris-setosa
...,...,...,...,...
147,6.7,5.2,2.3,Iris-virginica
148,6.3,5.0,1.9,Iris-virginica
149,6.5,5.2,2.0,Iris-virginica
150,6.2,5.4,2.3,Iris-virginica


In [11]:
# choose a value for the missing entry with `fillna`
clean_iris = iris.fillna(0)
clean_iris.loc[125:128]

# extra
average_sepal_width = iris['sepal width'].mean()
iris['sepal width'] = iris['sepal width'].fillna(average_sepal_width)

average_sepal_length = iris['sepal length'].mean()
iris['sepal length'] = iris['sepal length'].fillna(average_sepal_length)

display(iris.columns)
for name in iris.columns:
    if name != 'class':
        average_value = iris[name].mean()
        iris[name] = iris[name].fillna(average_value)

Index(['sepal length', 'sepal width', 'petal length', 'petal width', 'class'], dtype='object')

We used a couple of helper functions. [`dropna` will remove any rows with empty data](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html) while [`fillna` will replace any empty values with some other value](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html).

I tend not to fill in missing values, I tend to drop the whole row - that should be your default option but don't do it without paying attention to what is dropped and why.  Too many people just apply `dropna` without thinking.

# Exercise - find errors

Identify the erroneous data in the `class` column and remove that row.

**Advanced** Imagine you had not seen the error, what type of pandas code could you construct to find such an error.  I would suggest extracting that column as a series, get all the unique values in the series (`drop_duplicates` will help).

In [16]:
print("put your solution here")
display(clean_iris)
# print(clean_iris['class'].unique())
mask_no_6point3 = clean_iris['class'] != '6.3'
iris_class_clean = clean_iris[mask_no_6point3]
display(iris_class_clean)

put your solution here


,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
147,6.7,3.0,5.2,2.3,Iris-virginica
148,6.3,2.5,5.0,1.9,Iris-virginica
149,6.5,3.0,5.2,2.0,Iris-virginica
150,6.2,3.4,5.4,2.3,Iris-virginica


,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
147,6.7,3.0,5.2,2.3,Iris-virginica
148,6.3,2.5,5.0,1.9,Iris-virginica
149,6.5,3.0,5.2,2.0,Iris-virginica
150,6.2,3.4,5.4,2.3,Iris-virginica


# Exercise - fishing

**Advanced** Find any other erroneous data and fix it.

In [22]:
print("put your solution here")
print(clean_iris['sepal length'].unique())
print(clean_iris['sepal width'].unique())
print(clean_iris['petal length'].unique())
print(clean_iris['petal width'].unique())

mask_no_22 = clean_iris['petal width'] != 22
cleaned_iris = clean_iris[mask_no_22]

print("-- cleaned iris --")
display(cleaned_iris)
print("-- cleaned iris with duplicates removed --")
display(cleaned_iris.drop_duplicates())

put your solution here
[5.1 4.9 4.7 4.6 5.  5.4 4.4 4.8 4.3 5.8 5.7 5.2 5.5 4.5 5.3 7.  6.4 6.9
 6.5 6.3 6.6 5.9 6.  6.1 5.6 6.7 6.2 6.8 7.1 7.6 7.3 7.2 7.7 7.4 7.9]
[3.5 3.  3.2 3.1 3.6 3.9 3.4 2.9 3.7 4.  4.4 3.8 3.3 4.1 4.2 2.3 2.8 2.4
 2.7 2.  2.2 2.5 2.6 0. ]
[1.4 1.3 1.5 1.7 1.6 1.1 1.2 1.  1.9 4.7 4.5 4.9 4.  4.6 3.3 3.9 3.5 4.2
 3.6 4.4 4.1 4.8 4.3 5.  3.8 3.7 5.1 3.  6.  5.9 5.6 5.8 6.6 6.3 6.1 5.3
 5.5 6.7 6.9 5.7 6.4 5.4 5.2]
[ 0.2  0.4  0.3  0.1  0.5  0.6  1.4  1.5  1.3  1.6  1.   1.1  1.8  1.2
  1.7  2.5  1.9  2.1  2.2  2.   2.4  2.3 22. ]
-- cleaned iris --


,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
147,6.7,3.0,5.2,2.3,Iris-virginica
148,6.3,2.5,5.0,1.9,Iris-virginica
149,6.5,3.0,5.2,2.0,Iris-virginica
150,6.2,3.4,5.4,2.3,Iris-virginica


-- cleaned iris with duplicates removed --


,sepal length,sepal width,petal length,petal width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
147,6.7,3.0,5.2,2.3,Iris-virginica
148,6.3,2.5,5.0,1.9,Iris-virginica
149,6.5,3.0,5.2,2.0,Iris-virginica
150,6.2,3.4,5.4,2.3,Iris-virginica


# Exercise

There is a very useful method available on dataframes called "describe".  Below is an example of its use.

In [23]:
lithgow = pd.read_csv("data/rainfall/IDCJAC0009_047045_1800_Data.csv")
lithgow.describe()

,Bureau of Meteorology station number,Year,Month,Day,Rainfall amount (millimetres),Period over which rainfall was measured (days)
count,32293.0,32293.000000,32293.000000,32293.000000,31956.000000,5085.000000
mean,47045.0,1976.709101,6.506518,15.729136,0.699280,1.012389
std,0.0,25.523901,3.450379,8.800002,3.534565,0.149888
min,47045.0,1933.000000,1.000000,1.000000,0.000000,1.000000
25%,47045.0,1955.000000,4.000000,8.000000,0.000000,1.000000
50%,47045.0,1977.000000,7.000000,16.000000,0.000000,1.000000
75%,47045.0,1999.000000,10.000000,23.000000,0.000000,1.000000
max,47045.0,2021.000000,12.000000,31.000000,113.000000,7.000000


Experiment with this method on data you know well, [check the documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html).  How do you think this method can help you find erroneous data in your DataFrames?

# Concept Summary
  * We can index into a table with a mask
  * We can use boolean operators to create useful masks
  * `head`, `tail`, and `info` are useful for learning about your data
  * `dropna`, `drop_duplicates`, `duplicates`, `fillna` are usefull for cleaning your data

# Python concepts
  * `head`, `tail`, `info`, `dropna`, `drop_duplicates`, `duplicates`, `fillna` are all _methods_ on the data frame object
  * most of the methods we used returned entirely fresh values which we needed to capture in a new variable.  The original data frame was not changed by methods like `fillna`.
  * the `drop` method of a dataframe can take different parameters to do different things.  `index=` will drop a row, `columns=` will drop a column.  Both versions return a whole new DataFrame, leaving the original untouched.
